# Analyzing HHS hospital occupancy data

#### An analysis of HHS hospital data, by @datagraphics and @stiles.

#### **Questions?** [matt.stiles@latimes.com](matt.stiles@latimes.com) \\ 310.529.8749

---

### Import Python tools

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd
import jenkspy
import matplotlib.pyplot as plt

%matplotlib inline
import json
import numpy as np
import altair as alt
import altair_latimes as lat

alt.themes.register("latimes", lat.theme)
alt.themes.enable("latimes")
pd.options.display.max_columns = 50
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

---

### Define cities and regions metadata

In [3]:
metadata_df = pd.read_csv("input/city-metadata.csv", dtype={"fips": str})

In [4]:
metadata_df = metadata_df[["county", "fips", "population", "region"]]

In [5]:
socal = ["Los Angeles", "Orange", "Ventura", "San Bernardino", "Riverside"]
bayarea = [
    "Alameda",
    "Contra Costa",
    "Marin",
    "Napa",
    "San Francisco",
    "San Mateo",
    "Santa Clara",
    "Solano",
    "Sonoma",
]

### Get the latest url from the HHS API and read in the latest dataframe

In [6]:
# Data dictionary: https://healthdata.gov/covid-19-reported-patient-impact-and-hospital-capacity-facility-data-dictionary
# Data source: https://healthdata.gov/dataset/covid-19-reported-patient-impact-and-hospital-capacity-facility

In [7]:
metadata = pd.read_json(
    "https://healthdata.gov/api/3/action/package_show?id=d475cc4e-83cd-4c16-be57-9105f300e0bc&page=0"
).result[0]["resources"]

In [8]:
src = pd.read_csv(
    metadata[0]["url"],
    dtype={"fips_code": str, "zip": str},
    infer_datetime_format=True,
    parse_dates=True,
)

In [9]:
# src = pd.read_csv('https://healthdata.gov/sites/default/files/reported_hospital_capacity_admissions_facility_level_weekly_average_timeseries_20201228.csv',\
# dtype={'fips_code':str, 'zip':str}, infer_datetime_format=True, parse_dates=True)

In [10]:
df = src.copy()

In [11]:
ca_data = pd.DataFrame(df.loc[df.state == "CA"])

In [12]:
ca_data["fips"] = ca_data["fips_code"].str[2:5]

In [13]:
ca_timeseries = pd.merge(
    ca_data, metadata_df, how="left", left_on="fips", right_on="fips"
)

### What's the most recent collection week?

In [14]:
ca_timeseries["collection_week"] = pd.to_datetime(ca_timeseries["collection_week"])

In [15]:
ca_timeseries["collection_week"].max()

Timestamp('2021-01-01 00:00:00')

### Get rid of the -999999 suppressed values

In [16]:
ca_timeseries = ca_timeseries.replace([-999999.0], [0])

In [17]:
ca_timeseries["hospital_name"] = (ca_timeseries["hospital_name"]).str.title()
ca_timeseries["hospital_name"] = (
    (ca_timeseries["hospital_name"])
    .str.replace(" Of ", " of ")
    .str.replace("Hlth", "Health")
    .str.replace(" La", " LA")
)

---

## Calculate some hospitalization/covid rates

#### Calculate daily average of total staffed ICU beds

In [18]:
ca_timeseries["total_staffed_adult_icu_beds"] = round(
    (
        ca_timeseries["total_staffed_adult_icu_beds_7_day_sum"]
        / ca_timeseries["total_staffed_adult_icu_beds_7_day_coverage"]
    ),
    0,
)

#### Calculate daily average of occupied ICU beds

In [19]:
ca_timeseries["total_occupied_adult_icu_beds"] = round(
    (
        ca_timeseries["staffed_adult_icu_bed_occupancy_7_day_sum"]
        / ca_timeseries["staffed_adult_icu_bed_occupancy_7_day_coverage"]
    ),
    0,
)

#### Calculate daily average of COVID patients in the ICU

In [20]:
ca_timeseries["total_covid_icu_patients"] = round(
    (
        ca_timeseries[
            "staffed_icu_adult_patients_confirmed_and_suspected_covid_7_day_sum"
        ]
        / ca_timeseries[
            "staffed_icu_adult_patients_confirmed_and_suspected_covid_7_day_coverage"
        ]
    ),
    0,
)

#### Number of ICU beds available

In [21]:
ca_timeseries["total_available_adult_icu_beds"] = round(
    (
        ca_timeseries["total_staffed_adult_icu_beds"]
        - ca_timeseries["total_occupied_adult_icu_beds"]
    ),
    0,
)

#### Calculate daily ICU occupancy as percentage

In [22]:
ca_timeseries["pct_occupied_adult_icu_beds"] = round(
    (
        ca_timeseries["total_occupied_adult_icu_beds"]
        / ca_timeseries["total_staffed_adult_icu_beds"]
    ),
    2,
)

#### If a hospital reports 0 staffed adult ICU beds, drop them from the dataframe

In [23]:
filtered_timeseries = ca_timeseries[
    (ca_timeseries.total_staffed_adult_icu_beds_7_day_sum.notnull())
    & (ca_timeseries.total_staffed_adult_icu_beds_7_day_sum != 0)
].sort_values("total_staffed_adult_icu_beds_7_day_sum")

#### How many patients?

In [24]:
ca_timeseries["all_patients"] = round(
    (
        ca_timeseries["inpatient_beds_used_7_day_sum"]
        / ca_timeseries["inpatient_beds_used_7_day_coverage"]
    ),
    0,
)

#### How many Covid patients

In [25]:
ca_timeseries["total_adult_covid_patients"] = (
    ca_timeseries[
        "total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum"
    ]
    / ca_timeseries[
        "total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_coverage"
    ]
)

In [26]:
ca_timeseries["total_pediatric_covid_patients"] = (
    ca_timeseries[
        "total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_7_day_sum"
    ]
    / ca_timeseries[
        "total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_7_day_coverage"
    ]
)

In [27]:
ca_timeseries["total_covid_patients"] = round(
    ca_timeseries["total_adult_covid_patients"]
    + ca_timeseries["total_pediatric_covid_patients"],
    0,
).fillna(0)

#### What's the rate of Covid patients?

In [28]:
ca_timeseries["covid_patients_share"] = round(
    (ca_timeseries["total_covid_patients"] / ca_timeseries["all_patients"]), 2
)

In [29]:
ca_timeseries = ca_timeseries[ca_timeseries["all_patients"] > 0]

---

### Trim to the columns we want

In [30]:
trimmed_timeseries = ca_timeseries[
    [
        "hospital_name",
        "hospital_subtype",
        "ccn",
        "collection_week",
        "county",
        "fips",
        "total_staffed_adult_icu_beds",
        "total_covid_icu_patients",
        "total_available_adult_icu_beds",
        "pct_occupied_adult_icu_beds",
        "total_covid_patients",
        "all_patients",
        "covid_patients_share",
        "region",
    ]
].rename(
    columns={
        "hospital_name": "hospital",
        "collection_week": "week",
    }
)

---

### Filter the dataframe for Los Angeles County facilities

In [31]:
la = trimmed_timeseries[trimmed_timeseries["fips"] == "037"]
oc = trimmed_timeseries[trimmed_timeseries["fips"] == "059"]

### Filter the California dataframe to the most recent collection week

In [32]:
current_ca = trimmed_timeseries[
    trimmed_timeseries["week"] == trimmed_timeseries["week"].max()
].sort_values("covid_patients_share", ascending=False)

In [33]:
current_la = la[la["week"] == la["week"].max()].sort_values(
    "covid_patients_share", ascending=False
)

In [34]:
current_oc = oc[oc["week"] == oc["week"].max()].sort_values(
    "covid_patients_share", ascending=False
)

In [35]:
current_ca.to_csv("output/current_ca.csv", index=False)

### Specific place?

In [36]:
current_ca[current_ca["hospital"].str.contains("Methodist Hospital")].iloc[0]

hospital                          Methodist Hospital of Southern Ca
hospital_subtype                                         Short Term
ccn                                                          050238
week                                            2021-01-01 00:00:00
county                                                  Los Angeles
fips                                                            037
total_staffed_adult_icu_beds                                  21.00
total_covid_icu_patients                                      21.00
total_available_adult_icu_beds                                 1.00
pct_occupied_adult_icu_beds                                    0.95
total_covid_patients                                         143.00
all_patients                                                 211.00
covid_patients_share                                           0.68
region                                          Southern California
Name: 103, dtype: object

### Which CA hospitals have the greatest share of covid patients? 

In [37]:
current_ca[
    (current_ca["hospital_subtype"] != "Long Term")
    & (current_ca["total_covid_icu_patients"] > 0)
].sort_values("covid_patients_share", ascending=False).head(10)

,hospital,hospital_subtype,ccn,week,county,fips,total_staffed_adult_icu_beds,total_covid_icu_patients,total_available_adult_icu_beds,pct_occupied_adult_icu_beds,total_covid_patients,all_patients,covid_patients_share,region
237,Barstow Community Hospital,Short Term,050298,2021-01-01,San Bernardino,071,8.0,13.0,0.0,1.00,47.0,42.0,1.12,Southern California
102,Kaiser Foundation Hospital - Downey,Short Term,050139,2021-01-01,Los Angeles,037,58.0,52.0,0.0,1.00,231.0,256.0,0.90,Southern California
120,Kaiser Foundation Hospital - Baldwin Park,Short Term,050723,2021-01-01,Los Angeles,037,32.0,26.0,0.0,1.00,179.0,202.0,0.89,Southern California
125,Palmdale Regional Medical Center,Short Term,050204,2021-01-01,Los Angeles,037,24.0,14.0,5.0,0.79,117.0,137.0,0.85,Southern California
178,Garden Grove Hospital & Medical Center,Short Term,050230,2021-01-01,Orange,059,12.0,10.0,0.0,1.00,48.0,57.0,0.84,Southern California
211,Kaiser Foundation Hospital-Moreno Valley,Short Term,050765,2021-01-01,Riverside,065,29.0,19.0,0.0,1.00,76.0,90.0,0.84,Southern California
342,Santa Paula Hospital,Short Term,NaN,2021-01-01,Ventura,111,6.0,1.0,0.0,1.00,16.0,19.0,0.84,Southern California
206,"Kaiser Foundation Hospital, Riverside",Short Term,050686,2021-01-01,Riverside,065,48.0,46.0,0.0,1.00,167.0,200.0,0.84,Southern California
67,Community Hospital of Huntington Park,Short Term,050091,2021-01-01,Los Angeles,037,10.0,9.0,0.0,1.00,36.0,44.0,0.82,Southern California
229,Kaiser Foundation Hospital Fontana,Short Term,050140,2021-01-01,San Bernardino,071,122.0,77.0,15.0,0.88,404.0,498.0,0.81,Southern California


### Which facilities in LA have the greatest share of covid patients? 

In [38]:
current_la[
    (current_la["hospital_subtype"] != "Long Term")
    & (current_la["total_covid_icu_patients"] > 0)
].sort_values("covid_patients_share", ascending=False).head(10)

,hospital,hospital_subtype,ccn,week,county,fips,total_staffed_adult_icu_beds,total_covid_icu_patients,total_available_adult_icu_beds,pct_occupied_adult_icu_beds,total_covid_patients,all_patients,covid_patients_share,region
102,Kaiser Foundation Hospital - Downey,Short Term,050139,2021-01-01,Los Angeles,037,58.0,52.0,0.0,1.00,231.0,256.0,0.90,Southern California
120,Kaiser Foundation Hospital - Baldwin Park,Short Term,050723,2021-01-01,Los Angeles,037,32.0,26.0,0.0,1.00,179.0,202.0,0.89,Southern California
125,Palmdale Regional Medical Center,Short Term,050204,2021-01-01,Los Angeles,037,24.0,14.0,5.0,0.79,117.0,137.0,0.85,Southern California
67,Community Hospital of Huntington Park,Short Term,050091,2021-01-01,Los Angeles,037,10.0,9.0,0.0,1.00,36.0,44.0,0.82,Southern California
112,Kaiser Foundation Hospital - West LA,Short Term,050561,2021-01-01,Los Angeles,037,58.0,32.0,26.0,0.55,133.0,167.0,0.80,Southern California
92,Kaiser Foundation Hospital - Panorama City,Short Term,050137,2021-01-01,Los Angeles,037,47.0,35.0,8.0,0.83,152.0,202.0,0.75,Southern California
69,San Gabriel Valley Medical Center,Short Term,050132,2021-01-01,Los Angeles,037,18.0,15.0,0.0,1.00,63.0,84.0,0.75,Southern California
89,"Martin Luther King, Jr. Community Hospital",Short Term,050779,2021-01-01,Los Angeles,037,28.0,14.0,1.0,0.96,156.0,209.0,0.75,Southern California
133,San Dimas Community Hospital,Short Term,050588,2021-01-01,Los Angeles,037,21.0,20.0,0.0,1.00,49.0,65.0,0.75,Southern California
122,Alhambra Hospital Medical Center,Short Term,050281,2021-01-01,Los Angeles,037,13.0,10.0,0.0,1.00,51.0,68.0,0.75,Southern California


### Which facilities in OC have the greatest share of covid patients? 

In [39]:
current_oc[
    (current_oc["hospital_subtype"] != "Long Term")
    & (current_oc["total_covid_icu_patients"] > 0)
].sort_values("covid_patients_share", ascending=False).head(10)

,hospital,hospital_subtype,ccn,week,county,fips,total_staffed_adult_icu_beds,total_covid_icu_patients,total_available_adult_icu_beds,pct_occupied_adult_icu_beds,total_covid_patients,all_patients,covid_patients_share,region
178,Garden Grove Hospital & Medical Center,Short Term,050230,2021-01-01,Orange,059,12.0,10.0,0.0,1.00,48.0,57.0,0.84,Southern California
185,Kaiser Foundation Hospital - Orange County - A...,Short Term,050609,2021-01-01,Orange,059,85.0,67.0,1.0,0.99,254.0,351.0,0.72,Southern California
172,Ahmc Anaheim Regional Medical Center,Short Term,050226,2021-01-01,Orange,059,28.0,23.0,0.0,1.00,96.0,137.0,0.70,Southern California
193,Providence St. Jude Medical Center,Short Term,050168,2021-01-01,Orange,059,63.0,24.0,2.0,0.97,210.0,305.0,0.69,Southern California
184,Huntington Beach Hospital,Short Term,050526,2021-01-01,Orange,059,13.0,10.0,1.0,0.92,44.0,64.0,0.69,Southern California
171,Placentia Linda Hospital,Short Term,050589,2021-01-01,Orange,059,12.0,5.0,5.0,0.58,48.0,71.0,0.68,Southern California
192,La Palma Intercommunity Hospital,Short Term,050580,2021-01-01,Orange,059,9.0,8.0,0.0,1.00,29.0,43.0,0.67,Southern California
191,Providence St. Joseph Hospital,Short Term,050069,2021-01-01,Orange,059,42.0,29.0,2.0,0.95,235.0,354.0,0.66,Southern California
180,Fountain Valley Regional Hospital & Medical Ce...,Short Term,050570,2021-01-01,Orange,059,38.0,28.0,1.0,0.97,182.0,280.0,0.65,Southern California
174,Los Alamitos Medical Center,Short Term,050551,2021-01-01,Orange,059,17.0,17.0,0.0,1.00,110.0,179.0,0.61,Southern California


In [40]:
current_la["covid_patients_share"] = round(
    (current_la["covid_patients_share"] * 100), 2
)

### Output top LA hospitals for CMS table

In [41]:
current_la[
    (current_la["hospital_subtype"] != "Long Term")
    & (current_la["total_covid_icu_patients"] > 0)
][["hospital", "total_covid_patients", "covid_patients_share"]].sort_values(
    "covid_patients_share", ascending=False
).head(
    10
).to_csv(
    "output/current_la_hospitals.csv", index=False
)

### Which facilities in LA have the highest ICU capacity rates? 

In [42]:
current_la[(current_la["total_covid_icu_patients"] > 0)].sort_values(
    "pct_occupied_adult_icu_beds", ascending=False
).head(10)

,hospital,hospital_subtype,ccn,week,county,fips,total_staffed_adult_icu_beds,total_covid_icu_patients,total_available_adult_icu_beds,pct_occupied_adult_icu_beds,total_covid_patients,all_patients,covid_patients_share,region
137,Cedars-Sinai Medical Center,Short Term,050625,2021-01-01,Los Angeles,037,107.0,80.0,-39.0,1.36,336.0,830.0,40.0,Southern California
102,Kaiser Foundation Hospital - Downey,Short Term,050139,2021-01-01,Los Angeles,037,58.0,52.0,0.0,1.00,231.0,256.0,90.0,Southern California
96,Encino Hospital Medical Center,Short Term,050158,2021-01-01,Los Angeles,037,6.0,3.0,0.0,1.00,25.0,70.0,36.0,Southern California
120,Kaiser Foundation Hospital - Baldwin Park,Short Term,050723,2021-01-01,Los Angeles,037,32.0,26.0,0.0,1.00,179.0,202.0,89.0,Southern California
111,Providence Little Company of Mary Med Ctr Torr...,Short Term,050353,2021-01-01,Los Angeles,037,33.0,23.0,0.0,1.00,164.0,279.0,59.0,Southern California
75,Whittier Hospital Medical Center,Short Term,050735,2021-01-01,Los Angeles,037,25.0,17.0,0.0,1.00,69.0,127.0,54.0,Southern California
70,Beverly Hospital,Short Term,050350,2021-01-01,Los Angeles,037,25.0,19.0,0.0,1.00,81.0,136.0,60.0,Southern California
95,Northridge Hospital Medical Center,Short Term,050116,2021-01-01,Los Angeles,037,45.0,33.0,0.0,1.00,130.0,257.0,51.0,Southern California
65,Pacifica Hospital of The Valley,Short Term,050378,2021-01-01,Los Angeles,037,13.0,4.0,0.0,1.00,32.0,64.0,50.0,Southern California
115,Usc Verdugo Hills Hospital,Short Term,050124,2021-01-01,Los Angeles,037,15.0,10.0,0.0,1.00,63.0,126.0,50.0,Southern California


### How many LA County hospitals (with Covid patients) are in our dataframe? 

In [43]:
len(current_la[(current_la["total_covid_icu_patients"] > 0)])

73

### How many have 'high' ICU occupancy rates? 

In [44]:
len(current_la[current_la["pct_occupied_adult_icu_beds"] > 0.90])

49

### Top 10 Bay Area hospotals by Covid share? 

In [45]:
current_ca[current_ca["county"].isin(bayarea)].sort_values(
    "covid_patients_share", ascending=False
).head(10)

,hospital,hospital_subtype,ccn,week,county,fips,total_staffed_adult_icu_beds,total_covid_icu_patients,total_available_adult_icu_beds,pct_occupied_adult_icu_beds,total_covid_patients,all_patients,covid_patients_share,region
302,Kaiser Foundation Hospital-San Jose,Short Term,050604,2021-01-01,Santa Clara,085,20.0,16.0,2.0,0.90,97.0,159.0,0.61,Bay Area
299,Regional Medical Center of San Jose,Short Term,050125,2021-01-01,Santa Clara,085,47.0,29.0,0.0,1.00,118.0,242.0,0.49,Bay Area
289,Kaiser Foundation Hospital - South San Francisco,Short Term,050070,2021-01-01,San Mateo,081,13.0,6.0,1.0,0.92,43.0,89.0,0.48,Bay Area
6,Alameda Hospital,Short Term,050211,2021-01-01,Alameda,001,8.0,4.0,1.0,0.88,20.0,43.0,0.47,Bay Area
291,Seton Medical Center,Short Term,050289,2021-01-01,San Mateo,081,16.0,6.0,6.0,0.62,33.0,73.0,0.45,Bay Area
4,St Rose Hospital,Short Term,050002,2021-01-01,Alameda,001,15.0,7.0,4.0,0.73,25.0,57.0,0.44,Bay Area
11,Kaiser Foundation Hospital - San Leandro,Short Term,050777,2021-01-01,Alameda,001,19.0,13.0,1.0,0.95,73.0,166.0,0.44,Bay Area
287,San Mateo Medical Center,Short Term,050113,2021-01-01,San Mateo,081,7.0,5.0,0.0,1.00,17.0,40.0,0.42,Bay Area
23,Sutter Delta Medical Center,Short Term,050523,2021-01-01,Contra Costa,013,12.0,5.0,0.0,1.00,43.0,103.0,0.42,Bay Area
321,Healdsburg District Hospital,Critical Access Hospitals,051321,2021-01-01,Sonoma,097,5.0,0.0,5.0,0.00,4.0,10.0,0.40,Bay Area


---

## Get medical facilities' geographic data

In [46]:
# From HHS: https://maps3.arcgisonline.com/arcgis/rest/services/A-16/HHS_IOM_Health_Resources/MapServer

In [47]:
hospitals = gpd.read_file(
    "/Users/mhustiles/data/github/AGStoShapefile/backupdir/HHS_Hospital/HHS_Hospital_1608139617293.geojson"
)

In [48]:
medical_centers = gpd.read_file(
    "/Users/mhustiles/data/github/AGStoShapefile/backupdir/HHS_Medical_Center/HHS_Medical_Center_1608139616289.geojson"
)

### Clean up the column names so we can merge the facilities dataframes

In [49]:
facilities = [hospitals, medical_centers]

In [50]:
hospitals.rename(
    columns={
        "Name_new": "name",
        "Address_1": "address",
        "City": "city",
        "State_1": "state",
        "ZipCode": "zipcode",
        "PhoneNum": "phone",
        "County_Nam": "county",
        "Provider_N": "provider_id",
        "Hospital_T": "type",
        "Hospital_O": "operation",
        "Emergency_": "emergency",
    },
    inplace=True,
)

In [51]:
medical_centers.rename(
    columns={
        "Hospital_N": "name",
        "Address1": "address",
        "City_1": "city",
        "State_1": "state",
        "ZipCode": "zipcode",
        "PhoneNum": "phone",
        "County_Nam": "county",
        "Provider_N": "provider_id",
        "Hospital_T": "type",
        "Hospital_O": "operation",
        "Emergency_": "emergency",
    },
    inplace=True,
)

### Concatenate the different facility types into one dataframe, and also filter that just to CA

In [52]:
all_medical_geo = pd.concat(facilities)

In [53]:
all_medical_geo_ca = all_medical_geo[all_medical_geo["state"] == "CA"]

### We might be better off using HHS' own locations data with CCNs

In [54]:
locations = pd.read_csv(
    "input/hospital_locations.csv",
    dtype={"latitude": float, "longitude": float, "CCN": str},
)

In [55]:
locations.columns = (
    locations.columns.str.strip()
    .str.lower()
    .str.replace(" ", "_")
    .str.replace("(", "")
    .str.replace(")", "")
    .str.replace("-", "_")
)

/Users/mhustiles/.virtualenvs/notebooks-hnoN6uG0/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  """
/Users/mhustiles/.virtualenvs/notebooks-hnoN6uG0/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  


### Even though it has the wrong location for White Memorial. Let's fix.

In [56]:
locations.loc[
    (locations.facility_name == "WHITE MEMORIAL MEDICAL CENTER"), "longitude"
] = -118.2176219

In [57]:
locations.loc[
    (locations.facility_name == "WHITE MEMORIAL MEDICAL CENTER"), "latitude"
] = 34.0493044

### Convert the lon/lat fields into a geodataframe

In [58]:
locations = gpd.GeoDataFrame(
    locations, geometry=gpd.points_from_xy(locations.longitude, locations.latitude)
)

### And then confine it to California

In [59]:
locationsca = gpd.GeoDataFrame(locations[locations["state"] == "CA"])

### Make the CCN string match the hospital capacity data

In [60]:
locationsca["ccn"] = locationsca["ccn"].str.zfill(6)

---

### Merge with our CA medical facilities

In [61]:
current_ca_geo = locationsca.merge(current_ca, left_on="ccn", right_on="ccn")

In [62]:
len(current_ca_geo)

347

In [63]:
current_ca_geo.rename(
    columns={
        "county_x": "county",
        "city_x": "city",
        "address_x": "address",
        "state_x": "state",
    },
    inplace=True,
)

In [64]:
# current_ca_geo.drop(['county_y', 'hospital_subtype', 'provider_id', 'ccn', 'emergency', 'address', 'OBJECTID', 'state', 'name', 'phone', 'zipcode'], axis=1, inplace=True)

---

### Filter to just LA County hospitals

In [65]:
current_la_geo = current_ca_geo[current_ca_geo["fips"] == "037"]

### How many facilities in California?

In [66]:
len(current_ca_geo)

347

### How many in L.A. County?

In [67]:
len(current_la_geo)

84

---

### Export hospital specific data for maps and tables

In [68]:
current_ca_geo.to_file("output/current_ca_geo.geojson", driver="GeoJSON")

In [69]:
current_ca_geo.to_csv("output/current_ca_geo.csv", index=False)

In [70]:
current_la_geo.to_file("output/current_la_geo.geojson", driver="GeoJSON")

In [71]:
current_la_geo.to_csv("output/current_la_geo.csv", index=False)

---

## Aggregate

### What's the average share of Covid patients in hospitals statewide?

In [72]:
state = (
    current_ca.groupby(["week"])
    .agg({"total_covid_patients": "sum", "all_patients": "sum"})
    .reset_index()
)

In [73]:
state["covid_patients_share"] = round(
    (state["total_covid_patients"] / state["all_patients"]), 2
)

In [74]:
state.head()

,week,total_covid_patients,all_patients,covid_patients_share
0,2021-01-01,21970.0,51255.0,0.43


### Covid patients by county

In [75]:
counties = (
    trimmed_timeseries.groupby(["county", "week", "region"])
    .agg({"total_covid_patients": "sum", "all_patients": "sum"})
    .reset_index()
)

In [76]:
counties["covid_patients_share"] = round(
    (counties["total_covid_patients"] / counties["all_patients"]), 2
)

#### Which counties have the most Covid patients (in most recent week)?

In [77]:
counties[counties["week"] == counties["week"].max()].sort_values(
    "total_covid_patients", ascending=False
).head(5)

,county,week,region,total_covid_patients,all_patients,covid_patients_share
409,Los Angeles,2021-01-01,Southern California,8200.0,15660.0,0.52
662,Orange,2021-01-01,Southern California,2185.0,4533.0,0.48
799,San Bernardino,2021-01-01,Southern California,1781.0,3230.0,0.55
731,Riverside,2021-01-01,Southern California,1637.0,2860.0,0.57
822,San Diego,2021-01-01,Southern California,1618.0,4302.0,0.38


#### Which counties have the highest percentage of Covid patients (in most recent week)?

In [78]:
counties[counties["week"] == counties["week"].max()].sort_values(
    "total_covid_patients", ascending=False
).head(5)

,county,week,region,total_covid_patients,all_patients,covid_patients_share
409,Los Angeles,2021-01-01,Southern California,8200.0,15660.0,0.52
662,Orange,2021-01-01,Southern California,2185.0,4533.0,0.48
799,San Bernardino,2021-01-01,Southern California,1781.0,3230.0,0.55
731,Riverside,2021-01-01,Southern California,1637.0,2860.0,0.57
822,San Diego,2021-01-01,Southern California,1618.0,4302.0,0.38


### How has this changed over time? 

In [79]:
counties_grouped = (
    counties.groupby(["county", "week", "region"])
    .agg({"total_covid_patients": "sum", "covid_patients_share": "mean"})
    .reset_index()
)

In [80]:
la_grouped = pd.DataFrame(counties_grouped[counties_grouped["county"] == "Los Angeles"])

In [81]:
la_grouped.tail()

,county,week,region,total_covid_patients,covid_patients_share
405,Los Angeles,2020-12-04,Southern California,3666.0,0.26
406,Los Angeles,2020-12-11,Southern California,5038.0,0.34
407,Los Angeles,2020-12-18,Southern California,6497.0,0.43
408,Los Angeles,2020-12-25,Southern California,7584.0,0.50
409,Los Angeles,2021-01-01,Southern California,8200.0,0.52


In [82]:
la_grouped.to_csv("output/la_grouped.csv", index=False)

In [83]:
alt.Chart(la_grouped).mark_area().encode(
    x=alt.X("week:T", axis=alt.Axis(format="", title=" ", tickCount=3)),
    y=alt.Y(
        "covid_patients_share",
        title="Covid patients in LA County hospitals",
        axis=alt.Axis(format="%", tickCount=6),
    ),
).properties(width=500, height=500)

alt.Chart(...)

### Just southern California counties? 

In [84]:
socal_grouped = (
    counties_grouped[counties_grouped["county"].isin(socal)]
    .groupby(["county", "week"])
    .agg({"total_covid_patients": "sum", "covid_patients_share": "mean"})
    .reset_index()
)

In [85]:
alt.Chart(socal_grouped).mark_area().encode(
    x=alt.X("week:T", axis=alt.Axis(format="", title=" ", tickCount=3)),
    y=alt.Y(
        "covid_patients_share",
        title=" ",
        axis=alt.Axis(format="%", title="", tickCount=6),
    ),
    facet=alt.Facet("county"),
).properties(
    width=200, height=200, title="Share of all patients with Covid in SoCal Counties"
)

alt.Chart(...)

### Bay Area? 

In [86]:
bayarea_grouped = (
    counties_grouped[counties_grouped["county"].isin(bayarea)]
    .groupby(["week"])
    .agg({"total_covid_patients": "sum", "covid_patients_share": "mean"})
    .reset_index()
)

In [87]:
bayarea_grouped.tail()

,week,total_covid_patients,covid_patients_share
18,2020-12-04,1183.0,0.146667
19,2020-12-11,1515.0,0.182222
20,2020-12-18,1806.0,0.210000
21,2020-12-25,2001.0,0.235556
22,2021-01-01,2163.0,0.254444


In [88]:
alt.Chart(bayarea_grouped).mark_area().encode(
    x=alt.X("week:T", axis=alt.Axis(format="", title=" ", tickCount=3)),
    y=alt.Y(
        "covid_patients_share",
        title=" ",
        axis=alt.Axis(
            format="%",
            title="Share of all patients with Covid in Bay Area",
            tickCount=6,
        ),
    ),
).properties(width=500, height=500)

alt.Chart(...)

---

### What's going on in the most-populous counties 

In [89]:
counties_pop = pd.read_csv(
    "../census/processed/acs5_2018_population_counties.csv",
    dtype={"geoid": str, "population": int, "state": str, "county": str},
)

In [90]:
counties_pop.rename(columns={"universe": "population"}, inplace=True)

In [91]:
counties_pop.drop(
    ["universe_annotation", "universe_moe", "universe_moe_annotation"],
    axis=1,
    inplace=True,
)

In [92]:
ca_counties_pop = counties_pop[counties_pop["state"] == "06"]

In [93]:
big_ca_counties_pop = ca_counties_pop.sort_values("population", ascending=False).head(
    10
)

In [94]:
big_ca_counties_pop["name"] = big_ca_counties_pop["name"].str.replace(
    " County, California", ""
)

In [95]:
big_counties = list(big_ca_counties_pop.name)

In [96]:
big_counties_grouped = (
    counties_grouped[counties_grouped["county"].isin(big_counties)]
    .groupby(["county", "week", "region"])
    .agg({"total_covid_patients": "sum", "covid_patients_share": "mean"})
    .reset_index()
)

In [97]:
chart = (
    alt.Chart(big_counties_grouped)
    .mark_area()
    .encode(
        x=alt.X(
            "week:T", axis=alt.Axis(grid=False, format="%b.", title=" ", tickCount=4)
        ),
        y=alt.Y(
            "covid_patients_share",
            title=" ",
            axis=alt.Axis(
                format="%",
                title="",
                tickCount=5,
                gridColor="#dddddd",
                offset=6,
                tickSize=0,
                domainOpacity=0,
            ),
        ),
        facet=alt.Facet(
            "county",
            columns=5,
            title=" ",
            sort=alt.EncodingSortField(
                "covid_patients_share", op="max", order="descending"
            ),
        ),
        color=alt.Color("region", legend=alt.Legend(orient="top", title=" ")),
    )
    .properties(
        width=200,
        height=200,
        title="Share of all hospital patients with COVID-19 in large California counties",
    )
)

chart.configure_view(strokeOpacity=0)

alt.Chart(...)

---

### Regions

In [98]:
regions_timeseries = (
    trimmed_timeseries.groupby(["week", "region"])
    .agg({"total_covid_patients": "sum", "all_patients": "sum"})
    .reset_index()
)

In [99]:
regions_timeseries["covid_patients_share"] = round(
    (regions_timeseries["total_covid_patients"] / regions_timeseries["all_patients"]), 2
)

In [107]:
regions_timeseries.tail(5)

,week,region,total_covid_patients,all_patients,covid_patients_share
110,2021-01-01,Bay Area,2442.0,9117.0,0.27
111,2021-01-01,Greater Sacramento,969.0,3680.0,0.26
112,2021-01-01,Northern California,119.0,785.0,0.15
113,2021-01-01,San Joaquin Valley,2211.0,5243.0,0.42
114,2021-01-01,Southern California,16229.0,32430.0,0.50


In [101]:
chart = (
    alt.Chart(regions_timeseries)
    .mark_area()
    .encode(
        x=alt.X(
            "week:T", axis=alt.Axis(grid=False, format="%b.", title=" ", tickCount=4)
        ),
        y=alt.Y(
            "covid_patients_share",
            title=" ",
            axis=alt.Axis(
                format="%",
                title="",
                tickCount=5,
                gridColor="#dddddd",
                offset=6,
                tickSize=0,
                domainOpacity=0,
            ),
        ),
        facet=alt.Facet(
            "region",
            columns=5,
            title=" ",
            sort=alt.EncodingSortField(
                "covid_patients_share", op="max", order="descending"
            ),
        ),
        #     color=alt.Color('region', legend=alt.Legend(orient="top", title=' '))
    )
    .properties(
        width=200,
        height=200,
        title="Share of all hospital patients with COVID-19 by region",
    )
)

chart.configure_view(strokeOpacity=0)

alt.Chart(...)

---

### Hospital overcapacity

In [102]:
before = pd.read_csv("input/hospital_utilization.csv", dtype={"fac_no": str})

In [103]:
before[before["fac_name"].str.contains("KING")]

,fac_no,fac_name,fac_city,county,license_no,fac_zip,icu_beds,tot_lic_beds
185,106191230,"MARTIN LUTHER KING, JR. COMMUNITY HOSPITAL",LOS ANGELES,Los Angeles,060000132,90059,20,131


In [104]:
current_ca[current_ca["ccn"] == "050779"]

,hospital,hospital_subtype,ccn,week,county,fips,total_staffed_adult_icu_beds,total_covid_icu_patients,total_available_adult_icu_beds,pct_occupied_adult_icu_beds,total_covid_patients,all_patients,covid_patients_share,region
89,"Martin Luther King, Jr. Community Hospital",Short Term,050779,2021-01-01,Los Angeles,037,28.0,14.0,1.0,0.96,156.0,209.0,0.75,Southern California


In [105]:
before[["fac_name"]].sort_values("fac_name", ascending=False).to_csv(
    "output/before_names.csv", index=False
)

In [106]:
current_ca[["hospital"]].sort_values("hospital", ascending=False).to_csv(
    "output/current_names.csv", index=False
)